In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
brand_df = pd.read_csv('../data/all_brand_ratings.csv')

brand_df = brand_df.iloc[:, 1:]

brand_df.head()

,brand,es_rating,es_transparency,es_fair_labor,es_sustainably_made,sr_rating,sr_description,goy_rating,goy_planet,goy_people,goy_animals,goy_description,goyr_rating,goyr_planet,goyr_people,goyr_animals,goyr_description,total_ratings
0,KNOWN SUPPLY,1.7,1.7,5.0,1.7,4.0,"<a data-wpel-link=""external"" href=""https://kno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.850000
1,AMENDI,1.7,5.0,3.3,1.7,4.0,AMENDI is a brand that is making commendable e...,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0,4.0,AMENDI's environment rating is 'good'. It uses...,3.233333
2,SANVT,1.7,5.0,1.7,1.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000
3,Patagonia,5.0,5.0,5.0,5.0,4.0,Patagonia is a brand that focuses on sustainab...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,4.0,4.0,2.0,4.0,Our “Planet” rating evaluates brands based on ...,4.250000
4,Scotch & Soda,3.3,5.0,3.3,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.300000


In [3]:
brands = brand_df['brand']

In [ ]:
additional_brand_info = []
url = 'https://directory.goodonyou.eco/brand/'

for brand in brand_df['brand']:
    try:
        brand_converted = brand.lower() # lowercase
        brand_converted = brand_converted.replace(' ', '-') # replace spaces with dashes
        brand_converted = brand_converted.replace('&', 'and') # replace '&' with 'and'
        brand_converted = brand_converted.replace('+', '-') # replace '+' with '-'
        brand_converted = brand_converted.replace("'", '') # remove apostrophes
        brand_converted = brand_converted.replace('.', '') # remove periods
        brand_converted = brand_converted.replace('/', '') # remove slashes
        brand_converted = brand_converted.replace('é', 'e') # remove accent

        brand_url = url + brand_converted

        # get data on the brand's page
        response = requests.get(brand_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # check if Good On You does not have information about this brand
        h1_tag = soup.find('h1', class_='StyledHeading-sc-1rdh4aw-0 RBuGn')
        if h1_tag and h1_tag.text.strip() == 'Nothing to see here - 404':
            price_level = None
            location = None
            brand_url = None
        else:
            # get first 4 spans with this style (these are price signs)
            spans = soup.find_all('span', class_='StyledText-sc-1sadyjn-0 kkXGYR')[:4]
            
            # get number of dollar signs with opacity of 1
            # 1 = low cost, 4 = high cost
            price_level = len([span.get('style') for span in spans if 'opacity:1' in span.get('style')])

            # get brand location
            location_span = soup.find('span', string='location:')
            paragraph = location_span.find_parent('p')
            location = str(paragraph).split('<!-- -->')[1].split('</p>')[0]
    except:
        price_level = None
        location = None
        brand_url = None

    additional_brand_info.append((price_level, location, brand_url))

In [ ]:
# add information to brand's row
for i, brand in brand_df.iterrows():
    brand_df.loc[i, 'price_level'] = additional_brand_info[i][0]
    brand_df.loc[i, 'location'] = additional_brand_info[i][1]
    brand_df.loc[i, 'url'] = additional_brand_info[i][2]

In [20]:
brand_df.loc[:10, ['brand', 'price_level', 'location']]

,brand,price_level,location
0,KNOWN SUPPLY,2.0,United States
1,AMENDI,3.0,Sweden
2,SANVT,NaN,None
3,Patagonia,3.0,United States
4,Scotch & Soda,2.0,Australia
5,KINdom,NaN,None
6,Ecoalf,2.0,Spain
7,Unspun,3.0,United States
8,Pangaia,4.0,United Kingdom
9,Po-Zu,2.0,United Kingdom


In [13]:
# save additional information to CSV
brand_df.to_csv('../data/all_brand_ratings.csv', index=False)

In [12]:
# get distribution of price levels
# 1 = low cost, 4 = high cost
brand_df['price_level'].value_counts(normalize=True).sort_index()

price_level
1.0    0.075720
2.0    0.506173
3.0    0.320988
4.0    0.097119
Name: proportion, dtype: float64

In [11]:
# get distribution of brand ratings
brand_df['goy_rating'].value_counts(normalize=True).sort_index()

goy_rating
1.0    0.238095
2.0    0.495238
3.0    0.171429
4.0    0.066667
5.0    0.028571
Name: proportion, dtype: float64

In [10]:
# get distribution of countries
brand_df['location'].value_counts(normalize=True)

location
United States                                 0.302881
United Kingdom                                0.166255
Australia                                     0.120165
Germany                                       0.064198
Canada                                        0.040329
France                                        0.036214
Italy                                         0.035391
Sweden                                        0.027984
Netherlands                                   0.023045
Spain                                         0.021399
Denmark                                       0.021399
Japan                                         0.016461
Switzerland                                   0.012346
New Zealand                                   0.011523
Finland                                       0.010700
Lithuania                                     0.008230
Portugal                                      0.007407
India                                         0.007407
B